# Image Classification Using CNN with RestNet

In [ ]:
param = {
    'gpu_id' : '1',
    'model2_name' : 'vgg19_bn',
    'RWROP_categories': ['RWROP_0', 'RWROP_1'],
    'root_dir': './data',
    'model_dir': './model_parameters',
    'RWROP_testdata_dir': 'RWROP_testset',
    'd_vgg_features': 128, 
    'd_st_features': 256,
    'train_batch_size': [16,16,16,16,16], 
    'val_batch_size': 32,
    'test_batch_size': 32,
    'num_epochs': 100,
    'learning_rate': [0.0001,0.0001,0.0001,0.0001,0.0001],
    'S_S_GAP': [0.2,0.2,0.2,0.2,0.2],
    '3lossw': [[1, 0.5, 1],[1, 0.5, 1],[1, 0.5, 1],[1, 0.5, 1],[1, 0.5, 1]],
    'w_momentum': 0.9,
    'w_weight_decay': 0.0001,
    'workers': 4,
    'seed': 42,
    'loss-weight': [torch.tensor([1.30,1.70], dtype=torch.float32),torch.tensor([1.30,1.70], dtype=torch.float32),
                   torch.tensor([1.10,1.90], dtype=torch.float32),torch.tensor([1.25,1.75], dtype=torch.float32),
                   torch.tensor([1.15,1.85], dtype=torch.float32)],
}

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = param['gpu_id']

from torch import autograd

import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim
import torch.utils.data
from torchvision.io import read_image

import random
import scipy.io

import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

import torch.optim
import torch.utils.data
import pretrainedmodels
from vit_pytorch import ViT
from pytorch_pretrained_vit import ViT
# from lion_pytorch import Lion
import timm

import nni
from nni.experiment import Experiment


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(torch.cuda.is_available())

### Model information

In [ ]:
class VGGA_ST_stage(nn.Module):
    def __init__(self, VGG19, ST):
        super(VGGA_ST_stage, self).__init__()
        self.vgg_transform = transforms.Compose([transforms.Resize((224,224))])
        self.vgg19_backbone = nn.Sequential(*list(VGG19.children())[1:-7])
        ST.flatten = nn.ReLU()
        self.ST_backbone = ST
        self.MP = nn.MaxPool1d(4)
        self.vgg_linear3 = nn.Linear(6272,6272)
        nn.init.xavier_uniform(self.vgg_linear3.weight)
        self.vgg_linear4 = nn.Linear(6272,2048)
        nn.init.xavier_uniform(self.vgg_linear4.weight)
        self.vgg_linear5 = nn.Linear(2048,param['d_vgg_features'])
        nn.init.xavier_uniform(self.vgg_linear5.weight)
        self.vgg_linear6 = nn.Linear(param['d_vgg_features'],len(param['RWROP_categories']))
        nn.init.xavier_uniform(self.vgg_linear6.weight)
        self.vgg_softmax = nn.Softmax(dim=1)
        self.vgg_relu2 = nn.ReLU()
        self.vgg_relu3 = nn.ReLU()
        self.vgg_relu4 = nn.ReLU()
        self.st_linear1 = nn.Linear(1024,1024)
        nn.init.xavier_uniform(self.st_linear1.weight)
        self.st_relu1 = nn.ReLU()
        self.st_linear2 = nn.Linear(1024,param['d_st_features'])
        nn.init.xavier_uniform(self.st_linear2.weight)
        self.st_relu2 = nn.ReLU()
        self.st_linear3 = nn.Linear(param['d_st_features'],len(param['RWROP_categories']))
        nn.init.xavier_uniform(self.st_linear3.weight)
        self.st_softmax = nn.Softmax(dim=1)
        
        self.linear1 = nn.Linear((param['d_vgg_features']+param['d_st_features']),(param['d_vgg_features']+param['d_st_features']))
        nn.init.xavier_uniform(self.linear1.weight)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear((param['d_vgg_features']+param['d_st_features']),len(param['RWROP_categories']))
        nn.init.xavier_uniform(self.linear2.weight)
        self.softmax = nn.Softmax(dim=1)


    def resize_img_size(self, x):
        for img_idx in range(x.shape[0]):
            if img_idx == 0:
                y = self.vgg_transform(x[img_idx]).unsqueeze(0)
            else:
                y = torch.cat((y, self.vgg_transform(x[img_idx]).unsqueeze(0)),0)
        return y
 
    
    def forward(self, x):
        vgg_output = self.vgg19_backbone(self.resize_img_size(x))
        vgg_output = vgg_output.view(vgg_output.shape[0],-1)
        vgg_output = self.MP(vgg_output)
        vgg_output = self.vgg_linear3(vgg_output)
        vgg_output = self.vgg_relu2(vgg_output)
        vgg_output = self.vgg_linear4(vgg_output)
        vgg_output = self.vgg_relu3(vgg_output)
        vgg_output = self.vgg_linear5(vgg_output)
        vgg_output = self.vgg_relu4(vgg_output)
        vgg_output1 = self.vgg_linear6(vgg_output)
        vgg_output1 = self.vgg_softmax(vgg_output1)
        st_output = self.ST_backbone(x)
        st_output = self.st_linear1(st_output)
        st_output = self.st_relu1(st_output)
        st_output = self.st_linear2(st_output)
        st_output = self.st_relu2(st_output)
        st_output1 = self.st_linear3(st_output)
        st_output1 = self.st_softmax(st_output1)
        
        output = torch.cat((vgg_output, st_output),1)
        output = self.linear1(output)
        output = self.relu1(output)
        output = self.linear2(output)
        output = self.softmax(output)

        return output, vgg_output1, st_output1
    

### Test function

In [ ]:
def test(model, loader, test_prob_dir, test_label_dir, fold_AUC, Avg_Sensitivity, Avg_Specificity, Avg_Accuracy):

    with torch.no_grad():
        model.eval()
    
        for name, parameters in model.named_parameters():
            parameters.requires_grad = False

        fold_Sensitivity = []
        fold_Specificity = []
        fold_Accuracy = []
        num_correct, num_samples = 0, len(loader.dataset)

        test_batch_index = 0

        for x, y in loader:

            x_var = x.to(device)
            scores,resnet,st = model(x_var)
            del x_var
            _, preds = scores.data.cpu().max(1)
            if test_batch_index == 0:
                y_test = y.cpu().detach().numpy()
                y_probs = scores.cpu().detach().numpy()
                y_pred = preds.numpy()
            else:
                y_test = np.append(y_test, y.cpu().detach().numpy())
                y_probs = np.vstack((y_probs, scores.cpu().detach().numpy()))
                y_pred = np.append(y_pred, preds.cpu().detach().numpy())

            num_correct += (preds == y).sum()
            test_batch_index += 1

        acc = float(num_correct) / num_samples
        print(y_test.shape)
        y_probs = y_probs[...,1:]
        print(y_probs.shape)
        print(y_pred.shape)
    #     test_prob_dir = './data/eye_image/fold_0/Resnet_Aug_rwROSE_test_prob.txt'
        np.savetxt(test_prob_dir, y_probs ,fmt='%.10e', delimiter=" ") 
        np.savetxt(test_label_dir, y_test ,fmt='%.10e', delimiter=" ")

        print('Test accuracy: {:.2f}% ({}/{})'.format(
            100.*acc,
            num_correct,
            num_samples,
            ))

        AUC_score = roc_auc_score(y_test, y_probs, multi_class='ovr')
        print('AUC:')
        print(AUC_score)
        fold_AUC.append(AUC_score)
        conf_mat = confusion_matrix(y_test, y_pred)

        n_classes = conf_mat.shape[0]
        for i in range(n_classes):
            tp = conf_mat[i, i]
            fn = sum(conf_mat[i, :]) - tp
            fp = sum(conf_mat[:, i]) - tp
            tn = sum(sum(conf_mat)) - tp - fn - fp

            tpr = tp / (tp + fn)
            tnr = tn / (tn + fp)
            accuracy = (tp+tn) / (tp+fn+fp+tn)
            if i == 1:
                fold_Sensitivity.append(tpr)
                fold_Specificity.append(tnr)
                fold_Accuracy.append(accuracy)
    #         fold_Sensitivity.append(tpr)
    #         fold_Specificity.append(tnr)
    #         fold_Accuracy.append(accuracy)

            print(f"Class {i}: Sensitivity = {tpr:.5f}, Specificity = {tnr:.5f}, Accuracy = {accuracy:.5f}")
            print(len(fold_AUC))
            print(len(fold_Sensitivity))
            print(len(fold_Specificity))
            print(len(fold_Accuracy))
        Avg_Sensitivity.append(sum(fold_Sensitivity)/len(fold_Sensitivity))
        Avg_Specificity.append(sum(fold_Specificity)/len(fold_Sensitivity))
        Avg_Accuracy.append(sum(fold_Accuracy)/len(fold_Sensitivity))




    
    return acc

In [ ]:


    fold_AUC = []
    fold_Sensitivity = []
    fold_Specificity = []
    fold_Accuracy = []
    for i in range(5):
        RWROP_testdata_dir = os.path.join(param['root_dir'],  'fold_'+ str(i), param['RWROP_testdata_dir'])
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        test_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(RWROP_testdata_dir, transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=param['test_batch_size'], shuffle=False,
        num_workers=param['workers'], pin_memory=True)
        net = VGGA_ST_stage(pretrainedmodels.__dict__[param['model2_name']](num_classes=1000, pretrained='imagenet'), timm.create_model( 'swinv2_base_window16_256' , pretrained= True , num_classes=1024))
        net.to(device)
        for name, parameters in net.named_parameters():
            parameters.requires_grad = False
        net_dir = os.path.join(model_dir, 'VGGA-ST'+ 'fold_'+ str(i) +'_model_parameter.pkl')
        net.load_state_dict(torch.load(net_dir))
        test_prob_dir = os.path.join(param['root_dir'],  'fold_'+ str(i), 'VGGA-ST_test_prob.txt')
        test_label_dir = os.path.join(param['root_dir'],  'fold_'+ str(i), 'VGGA-ST_test_label.txt')
        test(net, test_loader, test_prob_dir, test_label_dir, fold_AUC, fold_Sensitivity, fold_Specificity, fold_Accuracy)
    print(len(fold_AUC))
    print(len(fold_Sensitivity))
    print(len(fold_Specificity))
    print(len(fold_Accuracy))
    AUC_result = np.array(fold_AUC)
    Sensitivity_result = np.array(fold_Sensitivity)
    Specificity_result = np.array(fold_Specificity)
    Accuracy_result = np.array(fold_Accuracy)
    print('AUC_result')
    print(AUC_result)
    print('Sensitivity_result')
    print(Sensitivity_result)
    print('Specificity_result')
    print(Specificity_result)
    print('Accuracy_result')
    print(Accuracy_result)

    print('Avg_AUC:')
    print(np.mean(AUC_result))
    
    print('std_AUC:')
    print(np.std(AUC_result))
    
    print('Avg_Sensitivity:')
    print(np.mean(Sensitivity_result))
    
    print('std_Sensitivity:')
    print(np.std(Sensitivity_result))
    
    print('Avg_Specificity:')
    print(np.mean(Specificity_result))
    
    print('std_Specificity:')
    print(np.std(Specificity_result))
    
    print('Avg_Accuracy:')
    print(np.mean(Accuracy_result))
    
    print('std_Accuracy:')
    print(np.std(Accuracy_result))
